In [ ]:
from google.colab import files

files.upload()

In [ ]:
! mkdir ~/.kaggle

! cp kaggle.json ~/.kaggle/

! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets download -d vishalmane109/indian-currency-note-images-dataset-2020

In [ ]:
! mkdir dataset

! unzip indian-currency-note-images-dataset-2020.zip -d dataset

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import os
%matplotlib inline

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Conv2D,MaxPooling2D,Dropout,Flatten
from pathlib import Path

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

data_agumentation=ImageDataGenerator(rescale=0./255,
                             shear_range=0.2,
                             zoom_range=0.2,
                             horizontal_flip=True,
                             vertical_flip=True

)

In [ ]:
train_data=data_agumentation.flow_from_directory(directory='/content/dataset/Indian currency dataset v1/training',
                                                 target_size=(256,256),
                                                 class_mode='categorical',
                                                batch_size=32
                                               )

In [ ]:
val_data=ImageDataGenerator().flow_from_directory(directory='/content/dataset/Indian currency dataset v1/validation',
                                                      target_size=(256,256),
                                                       class_mode='categorical'
                                                      )

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping

checkpoint=ModelCheckpoint("currency_detector_smal_model.h5", monitor='val_loss',verbose=1, save_best_only=True,save_weights_only=False,mode='auto', period=1)

early = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1, mode='auto')

In [ ]:
from tensorflow.keras.applications import ResNet50

# total number of classes (7 diffferent currency and 1 background class)
num_classes = 8

#define the mdel
my_new_model = Sequential()

# add the resnet to new defined model
my_new_model.add(ResNet50(include_top=False, pooling='avg',weights='imagenet'))

#last fully connected layer with softmax as a activation function
my_new_model.add(Dense(num_classes, activation='softmax'))

my_new_model.layers[0].trainable = False

In [ ]:
my_new_model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
my_new_model.fit_generator(
        train_data,
        epochs = 50,
        validation_data=val_data,
        callbacks=[checkpoint,early])

In [ ]:
model_json = my_new_model.to_json()
with open("resnet_50_model_epoch.json", "w") as json_file:
    json_file.write(model_json)

In [ ]:
class_labels = [
    '10','100','20','200','2000','50','500','Background'
]

In [ ]:
from tensorflow.keras.preprocessing import image

def prediction(file_name):
    img = image.load_img(file_name, target_size=(256,256))

    image_to_test = image.img_to_array(img)

    # Add a fourth dimension to the image (since Keras expects a list of images, not a single image)
    list_of_images = np.expand_dims(image_to_test, axis=0)

    # Make a prediction using the model
    results = my_new_model.predict(list_of_images)

    # Since we are only testing one image, we only need to check the first result
    single_result = results[0]
    print(single_result)
    # We will get a likelihood score for all 10 possible classes. Find out which class had the highest score.
    most_likely_class_index = int(np.argmax(single_result))
    class_likelihood = single_result[most_likely_class_index]

    # Get the name of the most likely class
    class_label = class_labels[most_likely_class_index]

    # Print the result
    print(file_name)
    print("This is image of a {} - Likelihood: {:2f}".format(class_label, class_likelihood))

In [ ]:
path = '/content/dataset/Indian currency dataset v1/test/500__17.jpg'

img = image.load_img(path, target_size=(256,256))
plt.imshow(img)

#predict the currecy

prediction(path)

In [ ]:
!pip install gTTS

#pyttsx3 is a text-to-speech conversion library in Python.
#Unlike alternative libraries, it works offline, and is compatible with both Python 2 and 3.

!pip install pyttsx3
!pip install pydub

In [ ]:
from gtts import gTTS
from tensorflow.keras.preprocessing import image
import os
import pyttsx3


def prediction(file_name):
    img = image.load_img(file_name, target_size=(256,256))

    image_to_test = image.img_to_array(img)

    #since Keras expects a list of images, not a single image,
    # Add a fourth dimension to the image

    list_of_images = np.expand_dims(image_to_test, axis=0)

    # Make a prediction using the model
    results = my_new_model.predict(list_of_images)

    # Since we are only testing one image, we only need to check the first result
    single_result = results[0]

    # We will get a likelihood score for all  possible classes.
    # Find out which class had the highest score.
    # the class with highest likelihood is predicted as the result.

    most_likely_class_index = int(np.argmax(single_result))
    class_likelihood = single_result[most_likely_class_index]

    # Get the name of the most likely class
    class_label = class_labels[most_likely_class_index]

    # Print the result
    print(file_name)
    print("This is image of a {} - Likelihood: {: .2f}".format(class_label, class_likelihood))

    # convert the actual prediction result text into audio file.
    tts(class_label,class_likelihood)

In [ ]:
from pydub import AudioSegment
import IPython

def tts(class_label,class_likelihood):
    language='en'

    # if no currency detected or uploaded image is  bagkground
    if(class_label=="Background"):

        mytext=' sorry but i am detecting only  the'+class_label+', please hold the note under the camera.'
    else:
        mytext="This is  {} Rs note, and I am  {: .2f} % sure of it".format(class_label, class_likelihood*100)

    # gTTS() converts text into the audio supports multiple languages.
    myobj = gTTS(text=mytext, lang=language, slow=False)

    #store audio result
    file='result.mp3'
    myobj.save(file)

In [ ]:
file_to_predict="/content/dataset/Indian currency dataset v1/test/20__350.jpg"

# display currency image
img = image.load_img(file_to_predict, target_size=(256,256))
plt.imshow(img)

#predict the currecy

prediction(file_to_predict)

In [ ]:
file='./result.mp3'

IPython.display.Audio(file)